<a href="https://colab.research.google.com/github/amit306/machineLearning/blob/main/BasicNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import math

In [ ]:
import random
import math

# --- Activation functions ---
def sigmoid(x):
    return 1.0 / (1.0 + math.exp(-x))

def sigmoid_derivative(y):
    return y * (1.0 - y)  # assumes input is already sigmoid(x)

# --- Neural Network Class ---
class NeuralNetwork:
    def __init__(self, num_inputs, num_hidden, num_outputs):
        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs

        # Random weights for input to hidden layer
        self.weights_input_hidden = [
            [random.uniform(-1, 1) for _ in range(self.num_hidden)]
            for _ in range(self.num_inputs)
        ]

        # Random weights for hidden to output layer
        self.weights_hidden_output = [
            [random.uniform(-1, 1) for _ in range(self.num_outputs)]
            for _ in range(self.num_hidden)
        ]

        # Biases
        self.bias_hidden = [random.uniform(-1, 1) for _ in range(self.num_hidden)]
        self.bias_output = [random.uniform(-1, 1) for _ in range(self.num_outputs)]

        # Learning rate
        self.lr = 0.5

    def forward(self, inputs):
        # Calculate hidden layer activations
        hidden_outputs = []
        for j in range(self.num_hidden):
            activation = self.bias_hidden[j]
            for i in range(self.num_inputs):
                activation += inputs[i] * self.weights_input_hidden[i][j]
            hidden_outputs.append(sigmoid(activation))

        # Calculate final output
        final_outputs = []
        for k in range(self.num_outputs):
            activation = self.bias_output[k]
            for j in range(self.num_hidden):
                activation += hidden_outputs[j] * self.weights_hidden_output[j][k]
            final_outputs.append(sigmoid(activation))

        return hidden_outputs, final_outputs

    def train(self, data, epochs=10000):
        for epoch in range(epochs):
            total_loss = 0.0  # Mean Squared Error for this epoch

            for inputs, targets in data:
                # Forward pass
                hidden_outputs, final_outputs = self.forward(inputs)

                # Output errors
                output_errors = [targets[k] - final_outputs[k] for k in range(self.num_outputs)]
                total_loss += sum((e ** 2 for e in output_errors)) / len(output_errors)

                # Hidden layer errors
                hidden_errors = [0.0] * self.num_hidden
                for j in range(self.num_hidden):
                    error = 0.0
                    for k in range(self.num_outputs):
                        error += output_errors[k] * self.weights_hidden_output[j][k]
                    hidden_errors[j] = error

                # Update weights: hidden to output
                for j in range(self.num_hidden):
                    for k in range(self.num_outputs):
                        delta = self.lr * output_errors[k] * sigmoid_derivative(final_outputs[k]) * hidden_outputs[j]
                        self.weights_hidden_output[j][k] += delta

                # Update output biases
                for k in range(self.num_outputs):
                    delta = self.lr * output_errors[k] * sigmoid_derivative(final_outputs[k])
                    self.bias_output[k] += delta

                # Update weights: input to hidden
                for i in range(self.num_inputs):
                    for j in range(self.num_hidden):
                        delta = self.lr * hidden_errors[j] * sigmoid_derivative(hidden_outputs[j]) * inputs[i]
                        self.weights_input_hidden[i][j] += delta

                # Update hidden biases
                for j in range(self.num_hidden):
                    delta = self.lr * hidden_errors[j] * sigmoid_derivative(hidden_outputs[j])
                    self.bias_hidden[j] += delta

            # Print loss every 1000 epochs
            if epoch % 1000 == 0:
                print(f"Epoch {epoch}: Loss = {total_loss:.6f}")

    def predict(self, inputs):
        _, outputs = self.forward(inputs)
        return outputs

# --- Training XOR ---
if __name__ == "__main__":
    # XOR truth table
    training_data = [
        ([0, 0], [0]),
        ([0, 1], [1]),
        ([1, 0], [1]),
        ([1, 1], [0])
    ]

    nn = NeuralNetwork(num_inputs=2, num_hidden=2, num_outputs=1)
    nn.train(training_data, epochs=10000)

    # Test predictions
    print("\n--- Predictions After Training ---")
    for inputs, _ in training_data:
        output = nn.predict(inputs)
        print(f"Input: {inputs} => Output: {output[0]:.4f}")


Epoch 0: Loss = 1.074533
Epoch 1000: Loss = 0.043105
Epoch 2000: Loss = 0.007839
Epoch 3000: Loss = 0.004230
Epoch 4000: Loss = 0.002888
Epoch 5000: Loss = 0.002190
Epoch 6000: Loss = 0.001763
Epoch 7000: Loss = 0.001475
Epoch 8000: Loss = 0.001268
Epoch 9000: Loss = 0.001112

--- Predictions After Training ---
Input: [0, 0] => Output: 0.0167
Input: [0, 1] => Output: 0.9852
Input: [1, 0] => Output: 0.9852
Input: [1, 1] => Output: 0.0165
